In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [8]:
# Set random seeds for reproducibility
torch.manual_seed(42)

In [9]:
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [11]:
df = pd.read_csv('/content/drive/MyDrive/Data/fmnist_small.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [12]:
df.shape

(6000, 785)

In [13]:
# train test split

X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
X_train = X_train/255.0
X_test = X_test/255.0

In [16]:
class CustomDataset(Dataset):

  def __init__(self, features, labels):

    # Convert to PyTorch tensors
    self.features = torch.tensor(features, dtype=torch.float32)
    self.labels = torch.tensor(labels, dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

In [17]:
train_dataset = CustomDataset(X_train, y_train)

In [18]:
test_dataset = CustomDataset(X_test, y_test)

In [19]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [20]:
len(train_loader)

150

In [21]:
class MyNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 10)

    )

  def forward(self, x):

    return self.model(x)


In [22]:
learning_rate = 0.1
epochs = 100

In [23]:
# instatiate the model
model = MyNN(X_train.shape[1])
model = model.to(device)
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.SGD(model.parameters(), lr=learning_rate)


In [24]:
# training loop

for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    # forward pass
    outputs = model(batch_features)

    # calculate loss
    loss = criterion(outputs, batch_labels)

    # back pass
    optimizer.zero_grad()
    loss.backward()

    # update grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')


Epoch: 1 , Loss: 1.3216368528207143
Epoch: 2 , Loss: 0.7793365333477656
Epoch: 3 , Loss: 0.6427524568637212
Epoch: 4 , Loss: 0.5751657398541768
Epoch: 5 , Loss: 0.5278772589564323
Epoch: 6 , Loss: 0.4953110021352768
Epoch: 7 , Loss: 0.4619268449147542
Epoch: 8 , Loss: 0.435526508440574
Epoch: 9 , Loss: 0.41890643616517387
Epoch: 10 , Loss: 0.39741519689559934
Epoch: 11 , Loss: 0.3866572337349256
Epoch: 12 , Loss: 0.37127780477205913
Epoch: 13 , Loss: 0.3490207275748253
Epoch: 14 , Loss: 0.34762982760866484
Epoch: 15 , Loss: 0.31585514172911644
Epoch: 16 , Loss: 0.3119578662017981
Epoch: 17 , Loss: 0.2958236836393674
Epoch: 18 , Loss: 0.2882651489476363
Epoch: 19 , Loss: 0.2712535789360603
Epoch: 20 , Loss: 0.25993844161430996
Epoch: 21 , Loss: 0.25771103213230767
Epoch: 22 , Loss: 0.2467066866407792
Epoch: 23 , Loss: 0.23920623461405435
Epoch: 24 , Loss: 0.22601548788448175
Epoch: 25 , Loss: 0.22358694796760878
Epoch: 26 , Loss: 0.21268802563349407
Epoch: 27 , Loss: 0.21945506637295087

In [25]:
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [26]:
# evaluation code
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.8241666666666667


In [27]:
# evaluation code
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.99875
